This Python file contains the code for web scrapping data about the MSIS program of SCU, Median pay for MSIS graduates.

Import all the required libraries.

In [1]:
import pandas
import requests

In [2]:
from bs4 import BeautifulSoup

Tried to web scrape data using LinkedIn, but could not do it. I realised late that we need to use Selenium becasue of the dynamic nature of the website. 

In [3]:
url = 'https://www.linkedin.com/school/10458916/'


In [4]:
r = requests.get(url)
##rtext = r.text()
soup = BeautifulSoup(r.content, "lxml")


In [5]:
for links in soup.find_all("a"):
    print(links.get("href"))

In [6]:
for links in soup.find_all("a"):
    print(links.get("href"),"  ", links)

In [7]:
url1 = 'https://www.linkedin.com/school/10458916/alumni/'
r1 = requests.get(url1)
r1.content
soup1 = BeautifulSoup(r1.content, "html5lib")


In [8]:
for links1 in soup1.find_all("a"):
    print(links1.get("href"))

Extracted data from SCU website to get a normal overview of the MSIS program.

In [9]:
url2 = 'https://www.scu.edu/business/ms-information-systems/prospective-students/class-profile/'
r2 = requests.get(url2)
r2.content
soup2 = BeautifulSoup(r2.content, "lxml")


In [10]:
for data1 in soup2.find_all("div", {"class" : "media-body"}):
    print(data1.text)


Class DemographicsWomen : 51%
Average age : 26
Multilingual :  89%
AcademicsAverage undergraduate GPA  3.2
Average GMAT  650
Average GRE  308
% holding graduate degrees  20%
Professional ProfileAverage work experience :  2.3 years
Employed at time of admission : 66%
Selected hiring companies :  Apple, Cisco, Ernst & Young, Facebook, NetApp, NVIDIA, Symantec, Twitter
Where SCU works


Tried to get data from a website called UberGrad.

In [11]:
url3 = 'https://www.ubergrad.com/universities/usa/mba-in-management-information-systems/top-universities'

r3 = requests.get(url3, auth = ('agasthya.agu@gmail.com','agasthya1992'))
soup3 = BeautifulSoup(r3.content, "lxml")



Tried using session to login to the website as the data I needed was restricted by a login access.

In [12]:
from lxml import html

session_requests = requests.session()

login_url = "https://www.ubergrad.com/login"
result = session_requests.get(login_url)
payload = {
	"ctl00$HTMLSection$ctl00$txtUsername": "agasthya.agu@gmail.com", 
	"ctl00$HTMLSection$ctl00$txtPassword": "agasthya1992", 
	"csrfmiddlewaretoken": "<CSRF_TOKEN>"
}
tree = html.fromstring(result.text)
#authenticity_token = list(set(tree.xpath("//input[@name='csrfmiddlewaretoken']/@value")))[0]
result = session_requests.post(
	login_url, 
	data = payload, 
	headers = dict(referer=login_url)
)

In [13]:
url = 'https://www.ubergrad.com/universities/usa/mba-in-management-information-systems/top-universities'
result = session_requests.get(
	url, 
	headers = dict(referer = url)
)

In [14]:
soup3 = BeautifulSoup(result.content, "lxml")



After trying normal methods, I tried using Selenium with a webdriver to get data. But was able to open the website, still faced login issues. I had spent a lot of time trying to fix this, but was unable to do so. So, I went back to trying to get data from a normal html website.

In [15]:
from selenium import webdriver
import os
path_to_chromedriver = '/Applications/chromedriver'
browser = webdriver.Chrome(path_to_chromedriver)
url = 'https://www.ubergrad.com/universities/usa/mba-in-management-information-systems/top-universities'
browser.get(url)


Needed median Pay information of people who work in MSIS related careers. This would give a view of how beneficial it would be to study a MSIS degree.

In [16]:
urlPay = 'https://www.payscale.com/research/US/Degree=Master_of_Science_(MS)%2C_Management_Information_Systems_(MIS)/Salary'
rPay = requests.get(urlPay)
rPay.content
soupPay = BeautifulSoup(rPay.content, "lxml")
#print(soup.prettify())

Saving the Data to arrays and later form DataFrames.

In [19]:
arrJobType = []
arrJobSalary = []
iCount = 0
for dataPay in soupPay.find_all("td"):
    if(iCount < 14):
        iCount = iCount+1
        if(iCount%2 != 0):
            arrJobType.append(dataPay.text)
        else:
            arrJobSalary.append(dataPay.text)

print(arrJobSalary)
print(arrJobType)

['$81,591', '$86,000', '$119,792', '$81,472', '$105,000', '$103,917', '$132,250']
['Project Manager, Information Technology (IT)', 'Software Engineer', 'Network Security Engineer', 'Software Developer', 'Senior Business Analyst', 'Senior Project Manager, IT', 'Information Technology (IT) Manager']


Get Data that shows the % job growth for the possible career tracks

In [21]:
urlPay10 = 'http://study.com/articles/Salary_and_Job_Information_for_MIS_Majors.html'
rPay10 = requests.get(urlPay10)
rPay10.content
soupPay10 = BeautifulSoup(rPay10.content, "lxml")


In [22]:
arrCareerTitles = []
arrProjectedGrowth = []
arrMedianSalary = []
iCount2 = 0
for dataPay10 in soupPay10.find_all("td"):
    iCount2 = iCount2+1
    if(iCount2 >1 and iCount2<=4):
        arrCareerTitles.append(dataPay10.text)
    elif(iCount2 > 9 and iCount2 <=12):
         arrProjectedGrowth.append(dataPay10.text)
    elif(iCount2 >13 and iCount2 <=16):    
         arrMedianSalary.append(dataPay10.text)
        


Get the specialization Tracks available in the MSIS program. This relates to the skills that is commonly expected of the students. 

In [23]:
urlTracks = 'https://www.scu.edu/business/ms-information-systems/curriculum/specialization-tracks/'
rTracks = requests.get(urlTracks)
rTracks.content
soupTracks = BeautifulSoup(rTracks.content, "lxml")

In [25]:
arrTracks = []

In [26]:
for dataTracks in soupTracks.find_all("h4"):
    arrTracks.append(dataTracks.text)    

In [27]:
dataframe=pandas.DataFrame(arrTracks,columns=['Program Tracks'])


In [28]:
data =  {'Job Type':arrJobType,'Salary':arrJobSalary}
dataframe2=pandas.DataFrame(data)


In [29]:
data2 =  {'Career Titles':arrCareerTitles,'Projected Growth Rate:(2014-2024)':arrProjectedGrowth, 'Median Salary':arrMedianSalary}
dataframe3=pandas.DataFrame(data2)


In [30]:
DataComp = requests.get('http://study.com/articles/Top_Ranked_School_for_Information_Technology_-_San_Jose_CA.html')
DataComp.content
soupDataComp = BeautifulSoup(DataComp.content, 'lxml')


In [31]:
DataComp = requests.get('http://study.com/articles/Top_Ranked_School_for_Information_Technology_-_San_Jose_CA.html')
DataComp.content
soupDataComp = BeautifulSoup(DataComp.content, 'lxml')
table1 = soupDataComp.find('table')


In [32]:
tableData1 = soupDataComp.find('table')
tableDataF = tableData1.find('tbody')
for rows in tableDataF.find_all('tr'):
    for cells in rows.find_all('td'):
        print(cells.text)     

 Rankings (2013) 
 Ranked 2nd among universities in the Western region* 
 Named the 9th best public school in the Western region* 
 106th among national universities in the U.S.*
 Location 
 Santa Clara, CA (about 2 miles from San Jose) 
 San Jose, CA 
 San Francisco, CA (roughly 42 miles from San Jose
 School Type 
 4-year, private not-for-profit 
 4-year, public 
 4-year, private not-for-profit
 Tuition & Fees (2012-2013) 
 $23,976 (graduate)** 
 $6,852, in-state; $18,510, out-of-state (undergrad)** 
 $965 per credit (bachelor's completion program)***; $20,390 per year (graduate)**
 Information Technology Degrees and Programs Offered 
 Master's degree 
 Bachelor's degree 
 Adult bachelor's completion and master's degrees
 Information Technology Degrees Awarded (All Levels) in 2011-2012 
 22** 
 1,242 for business administration and management (information technology not provided)** 
 248 for business administration and management (information technology not provided)**
 Most Popular 

In [33]:
dataCompF = {
    'Category' : [],
    'Santa Clara University' : [],
    'San Jose State University' : [],
    'University of San Francisco' : []
}

table_columns = ['Category','Santa Clara University','San Jose State University','University of San Francisco']
count = 0
for rows in tableDataF.find_all('tr'):
    for cells in rows.find_all('td'):
        dataCompF[table_columns[count%4]].append(cells.get_text() )
        count = count+1
    
DataCompFinal = pandas.DataFrame(dataCompF)  

In [131]:
dataframe.to_csv('Lab4-Dataset2.csv', index=False, encoding='utf-8')
dataframe2.to_csv('Lab4-Dataset3.csv', index=False, encoding='utf-8')
dataframe3.to_csv('Lab4-Dataset4.csv', index=False, encoding='utf-8')
DataCompFinal.to_csv('Lab4-Dataset5.csv', index=False, encoding='utf-8')

Santa Clara offers a good MSIS course!

- It has several industry oriented tracks which helps the students to keep in touch with the latest technologies used in the Industry. The curriculum is designed in consultation with Industry experts. This gives an advantage for the students, they can perform better due to the exposure to latest technology. 

- Ranked 2nd in the among the universities in the West region. This certifies for the level of the college, quality of the faculty and the level of the students that pursue the course in the university. 

- It has a good Return on Investment. The ROI for the program is quite high, considering the opportunities that are available after graduating from the program and the median pay for these opportunities. The number of jobs available for this particular course is also higher than the average job growth rate, this also adds up as an advantage for the course as this gives the students more opportunities. 


To solidify the claim even more, if we have more data on the actual number of students (alumni) who secured a job within six months of graduation from the university, it would be helpful. This would further solidify the claim that the MSIS course is good and offers the required exposure to the students in joining the work force of the Industry.


Link to tableau public Old viz - https://public.tableau.com/profile/aashreya.agasthya.kundurthy#!/vizhome/lab4_9/Dashboard1?publish=yes

Link to tableau public Redesigned -

https://public.tableau.com/profile/aashreya.agasthya.kundurthy#!/vizhome/Assignment-8_0/MSISSCU?publish=yes